<h1>Contaminación en Madrid</h1>

### Lectura y tratamiento

Leer y tratar los datos de mediciones de NO2 en Madrid, dándoles una forma más cómoda para su análisis.

In [97]:
import os
import re
import pandas as pd
import numpy as np

Extracción de datos

In [28]:
# Descomprimir origin (ZIP) a destination (directorio)
def unzip(origin, destination):
    !unzip $origin -d $destination

# Comprueba si el directorio dado existe y contiene ficheros
# Si no, lo rellena descomprimiendo
def setup_dir(filename):
    name = filename.split('.')[0]
    
    # Y si el directorio no existe?
    if not os.path.isdir(name): 
        os.mkdir(name)
        unzip(filename, name)
    else:
        # Y si el directorio está vacío?
        if not len(os.listdir(name)): 
            unzip(filename, name)
    
    return [os.path.join(name, file) for file in os.listdir(name)]

# Nos aseguramos de que todos los ficheros están descomprimidos y en el directorio esperado
files = setup_dir('Anio201810.zip')

Una vez tenemos la lista de ficheros a cargar, los cargamos a un solo dataframe:

In [99]:
extension = 'csv' # vienen en varios formatos, aquí definimos cual vamos a cargar
sep = ';'
remove_cols = ['MAGNITUD', 'PUNTO_MUESTREO'] # columnas prescindibles
magnitud = 8 # solo nos interesa el NO2 (MAGNITUD == 8)

# utilizo solo los ficheros con la extensión dada
relevant_files = [file for file in files if re.search(rf"\.{extension}$", file)]

# Carga un fichero y lo prepara
def cargar(filename):
    # Lectura
    df = pd.read_csv(filename, sep = sep)
    
    # Selección datos NO2
    df = df.loc[df.MAGNITUD == magnitud]
    
    # Eliminación de columnas poco relevantes
    df = df.drop(remove_cols, axis=1)
    return df

In [127]:
# Cargamos todos los ficheros a df
df = cargar(relevant_files[0])

if len(relevant_files) > 1:
    for file in relevant_files[1:]:
        new_df = cargar(file)
        df = df.append(new_df)

df

,PROVINCIA,MUNICIPIO,ESTACION,ANO,MES,DIA,H01,V01,H02,V02,...,H20,V20,H21,V21,H22,V22,H23,V23,H24,V24
90,28,79,4,2018,4,1,21.0,V,19.0,V,...,46.0,V,57.0,V,82.0,V,85.0,V,77.0,V
91,28,79,4,2018,4,2,67.0,V,45.0,V,...,22.0,V,26.0,V,38.0,V,15.0,V,15.0,V
92,28,79,4,2018,4,3,14.0,V,9.0,V,...,23.0,V,27.0,V,30.0,V,23.0,V,20.0,V
93,28,79,4,2018,4,4,8.0,V,3.0,V,...,14.0,V,19.0,V,22.0,V,28.0,V,23.0,V
94,28,79,4,2018,4,5,20.0,V,23.0,V,...,45.0,V,68.0,V,99.0,V,93.0,V,84.0,V
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4405,28,79,60,2018,9,26,16.0,V,13.0,V,...,47.0,V,26.0,V,17.0,V,15.0,V,13.0,V
4406,28,79,60,2018,9,27,9.0,V,9.0,V,...,28.0,V,47.0,V,71.0,V,109.0,V,74.0,V
4407,28,79,60,2018,9,28,86.0,V,33.0,V,...,26.0,V,73.0,V,120.0,V,129.0,V,116.0,V
4408,28,79,60,2018,9,29,62.0,V,26.0,V,...,20.0,V,52.0,V,93.0,V,97.0,V,102.0,V


Marcamos los datos no validados (la columna VX es distinta de 'V'):

In [128]:
horas = ["H%02d" % h for h in range(1, 25)]
vals = ["V%02d" % h for h in range(1, 25)]

for i, hora in enumerate(horas):
    val = vals[i]
    df[hora] = np.where(df[val] == 'V', df[hora], np.nan)

In [129]:
df = pd.melt(
    df,
    id_vars = ['PROVINCIA', 'MUNICIPIO', 'ESTACION', 'ANO', 'MES', 'DIA'],
    value_vars = horas,
    var_name = 'HORA',
    value_name = 'MEDIDA_NO2'
)

# Conservo solo los valores validados (no nulos)
df = df[df.MEDIDA_NO2.notnull()]
df = df.reset_index()

df

,index,PROVINCIA,MUNICIPIO,ESTACION,ANO,MES,DIA,HORA,MEDIDA_NO2
0,0,28,79,4,2018,4,1,H01,21.0
1,1,28,79,4,2018,4,2,H01,67.0
2,2,28,79,4,2018,4,3,H01,14.0
3,3,28,79,4,2018,4,4,H01,8.0
4,4,28,79,4,2018,4,5,H01,20.0
...,...,...,...,...,...,...,...,...,...
209076,209995,28,79,60,2018,9,26,H24,13.0
209077,209996,28,79,60,2018,9,27,H24,74.0
209078,209997,28,79,60,2018,9,28,H24,116.0
209079,209998,28,79,60,2018,9,29,H24,102.0


In [130]:
df['HORA'] = df.HORA.str.replace('H', '')
df['FECHAHORA'] = pd.to_datetime({'year': df.ANO, 'month': df.MES, 'day': df.DIA, 'hour': df.HORA})
df = df[['PROVINCIA', 'MUNICIPIO', 'ESTACION', 'FECHAHORA', 'MEDIDA_NO2']]
df = df.sort_values(by = 'FECHAHORA')
df

,PROVINCIA,MUNICIPIO,ESTACION,FECHAHORA,MEDIDA_NO2
2569,28,79,40,2018-01-01 01:00:00,12.0
2786,28,79,56,2018-01-01 01:00:00,37.0
2848,28,79,58,2018-01-01 01:00:00,1.0
2755,28,79,55,2018-01-01 01:00:00,9.0
2724,28,79,54,2018-01-01 01:00:00,8.0
...,...,...,...,...,...
202544,28,79,60,2019-01-01 00:00:00,78.0
201831,28,79,4,2019-01-01 00:00:00,70.0
202265,28,79,48,2019-01-01 00:00:00,91.0
202110,28,79,36,2019-01-01 00:00:00,83.0


In [125]:
df.describe(include = 'all', datetime_is_numeric=True)

,PROVINCIA,MUNICIPIO,ESTACION,FECHAHORA,MEDIDA_NO2
count,209081.0,209081.0,209081.000000,209081,209081.000000
mean,28.0,79.0,37.767358,2018-07-02 10:52:11.750852864,36.546669
min,28.0,79.0,4.000000,2018-01-01 01:00:00,1.000000
25%,28.0,79.0,24.000000,2018-04-02 03:00:00,15.000000
50%,28.0,79.0,40.000000,2018-07-02 10:00:00,29.000000
75%,28.0,79.0,55.000000,2018-10-01 18:00:00,51.000000
max,28.0,79.0,60.000000,2019-01-01 00:00:00,331.000000
std,0.0,0.0,17.626441,NaN,28.103390


A partir de aquí, se podría crear, a partir de df, una tabla en la BBDD (o añadirlo a una tabla ya existente).

### Arquitectura

Propongo una arquitectura para la actualización de los datos en tiempo real y su servicio a través de una API y un front-end.

<img src="esquema.png" alt="esquema"></img>

Configuraríamos algún software de orquestación de tareas, como Celery, que se encargaría de ejecutar periódicamente un script. Este programa, mediante HTTP, descargaría los datos actualizados tanto del Ayuntamiento de Madrid como de nuestra API meteorológica y los limpiaría, siguiendo el proceso que hemos escrito más arriba en este Notebook. Estos datos ya limpios servirían para actualizar la base de datos. A su vez, esta base de datos también podría recibir cargas manuales de datos cuando fuera necesario. El equipo de data science podría realizar consultas directamente a esta base de datos.

Adicionalmente, crearíamos un servidor web que serviría una API. Mediante esta API, el servidor gestionaría solicitudes de usuarios, ya sea directamente o través de la interfaz front-end, y devolvería los datos pertinentes de la base de datos.